## 环境
IDEA推荐直接使用[Codespaces](codespaces)

手机上抓包的话推荐使用[Stream](https://apps.apple.com/hk/app/stream/id1312141691)

安装依赖如果是用GitHub的Codespaces或者自己的电脑 只需首次安装，就是下一格的命令

In [ ]:
# 安装依赖
%pip install -r requirements.txt

## 第一步
需要用抓包工具抓取mail app中的请求，拿到请求头的User-Id和Access-Token
填入下一个格中的userId、accessToken

`User-Id和Access-Token相当于账号密码  切勿泄露`

In [1]:
# 这里填写User-Id和Access-Token的值
userId = ""  # @param {type:"string",description:""}
accessToken = "" # @param {type:"string",description:""}


# 这个request header是我抓取的hkt48 mail的请求，理论上48系其他团修改对应的名称`header['Host']`和请求域名`baseUrl`，没试过 我猜的
# 安卓修改Os-Type为ANDROID 
# 安卓不能直接用，要改下面的请求头内容，手上没有安卓手机，调试不了，请自行修改调试
# 以实际抓包显示的request header为准
header = {
    'Host': 'api-hkt.akb-groupmail.com',
    'Application-Version': '1.3.4',
    'Accept': '*/*',
    'Os-Type': 'IOS',
    'Terms-Version': '5',
    'User-Id': userId,
    'Access-Token': accessToken,
}
# 请求域名 按需修改
baseUrl = "https://api-hkt.akb-groupmail.com/"

## 第二步
获取mail列表和mail详情
列表保存为json格式 (项目目录下/data/metadata/page/)
详情保存为html (项目目录下/data/metadata/detail/)

In [2]:
import urllib.request
import urllib.parse
import json
import os
import datetime

# 获取详情
def getDetail(url,time):
    request = urllib.request.Request(url,headers=header)
    response = urllib.request.urlopen(request).read()
    # jsonData = json.loads(response)
    id = url.split("/")[-1]
    # 定义需要创建的目录和文件名
    dir_path = "./data/metadata/detail/"
    file_name = id+"_"+time+".html"
    file_path = os.path.join(dir_path, file_name)
    # 如果目录不存在，则创建目录
    if not os.path.isdir(dir_path):
        os.makedirs(dir_path)
    # 写入源文件
    fh = open(file_path,"wb")
    fh.write(response)
    fh.close()

# 获取page
def getList(page):
    request = urllib.request.Request(url+page,headers=header)
    response = urllib.request.urlopen(request).read()
    jsonData = json.loads(response)

    # 定义需要创建的目录和文件名
    dir_path = "./data/metadata/page/"
    file_name = page+".json"
    file_path = os.path.join(dir_path, file_name)

    # 如果目录不存在，则创建目录
    if not os.path.isdir(dir_path):
        os.makedirs(dir_path)

    # 写入原始数据
    fh = open(file_path,"wb")
    fh.write(response)
    fh.close()
    # 解析详情
    count = jsonData["mail_count"]
    for i in range(count):
        data =jsonData['mails'][i]
        # print(data)
        if(data['member']['id']!=0):
            # 将时间字符串解析为datetime对象
            time_str = data['receive_time']
            datetime_obj = datetime.datetime.strptime(time_str, "%Y/%m/%d %H:%M")
            # 将datetime对象格式化为指定的字符串格式
            formatted_time_str = datetime_obj.strftime("%Y_%m_%d_%H_%M")
            getDetail(data['detail_url'],formatted_time_str)
    # 判断还有没有下一页
    if not jsonData['has_next_page']:
        return False
    return True

# mail列表页
url = baseUrl+'v1/inbox?is_information=0&page='

for page in range(1, 30):
    if getList(str(page))==False:break



# 第三步
解析mail详情的html
下载图片  
    (项目目录下/data/image/)  
    (项目目录下/data/metadata/image/)
读取内容
    保存为变量 后面可选择导出为csv或json

In [4]:
from bs4 import BeautifulSoup
import re
import requests
def list_files(path):
    # 列出指定目录下的所有文件
    files = [f for f in os.listdir(path) if os.path.isfile(os.path.join(path, f))]
    # 按文件名排序
    files = sorted(files)
    return files
detailPath = "./data/metadata/detail"

# 按原路径保存图片
def saveImages(url_list):
    # 循环遍历链接数组
    for url in url_list:
        # 下载图片
        response = requests.get(url)
        # 如果响应状态码为200
        if response.status_code == 200:
            # 解析出路径和文件名
            directory, filename = url.rsplit('/', 1)
            directory = directory.replace('https://img.akb-groupmail.com/', '')
            path = os.path.join('./data/metadata/image', directory, filename)
            # 如果路径不存在，则创建路径
            if not os.path.exists(os.path.dirname(path)):
                os.makedirs(os.path.dirname(path))
            
            # 将图片保存到本地
            with open(path, 'wb') as f:
                f.write(response.content)
            # print('保存成功：', path)
        else:
            print('下载失败：', url)

# 循环调用该方法 将图片全部存入同一个目录 按日期和时间命名
def download_image(url, filename):
    try:
        response = requests.get(url)
        if response.status_code == 200:
            path = os.path.join('./data/image', filename)
            # 如果路径不存在，则创建路径
            if not os.path.exists(os.path.dirname(path)):
                os.makedirs(os.path.dirname(path))
            with open(path, 'wb') as f:
                f.write(response.content)
            # print('图片保存成功:', path)
        else:
            print('图片下载失败:', url)
    except Exception as e:
        print('图片下载异常:', e)


# 列出指定目录下的所有文件并按文件名排序输出
list = list_files(detailPath)
mapList = []
for file in list:
    # 从文件名解析id
    id = file.split('_')[0]
    # 从文件名解析时间
    time_pattern = r"\d{4}_\d{2}_\d{2}_\d{2}_\d{2}"
    match = re.search(time_pattern, file)
    time_str = ""
    if match:
        time_str = match.group()
        time =  datetime.datetime.strptime(time_str, "%Y_%m_%d_%H_%M")
    else:
        print("No match")
    html = os.path.join(detailPath, file)
    # 读取 HTML 文件
    with open(html, 'r', encoding='utf-8') as f:
        html_doc = f.read()
    # 使用 BeautifulSoup 解析 HTML 文件
    soup = BeautifulSoup(html_doc, 'html.parser')
    # 获取标题
    title = soup.title.string
    # 获取页面中的文字内容
    body = soup.body.body
    text = body.prettify()
    for tag in body.find_all('img'):
        index = text.find(str(tag))  # 获取标签在文本中的位置
        if index >= 0:
            text = text[:index] + f'[图片：{tag["src"]}]' + text[index+len(str(tag)):] # 在文本中替换标签
    # 剔除标签
    text = re.sub(r'<[^>]+>', '', text)
    # 剔除多余换行符
    text = re.sub(r'\n+', '\n', text)
    imgs = soup.find_all('img')
    srcList = []
    if imgs:
        for index,img in enumerate(imgs):
            srcList.append(img['src'])
            filename=file+"_"+str(index)+".jpg"
            download_image(img['src'],time_str+"_"+str(index)+".jpg")
        # print(srcList)
    # 按原路径保存图片
    saveImages(srcList)
    map={
        "id":id,
        "time":time.strftime('%Y-%m-%d %H:%M:%S'),
        "title":title,
        "text":text,
        "images":srcList
    }
    mapList.append(map)

## 保存为csv文件
文件保存在项目目录下/data/output.csv

In [5]:
import csv
output_file = "output.csv"
path = os.path.join('./data/', output_file)
# 如果路径不存在，则创建路径
if not os.path.exists(os.path.dirname(path)):
    os.makedirs(os.path.dirname(path))

with open(path, "w", newline="", encoding="utf-8-sig") as f:
    fields = ["id", "time", "title","text","images"]
    csv_writer = csv.DictWriter(f, fieldnames=fields)
    csv_writer.writeheader()
    csv_writer.writerows(mapList)

## 保存为json文件
文件保存在项目目录下/data/output.json

In [6]:
import json
output_file = "output.json"
json_str = json.dumps(mapList, ensure_ascii=False)
path = os.path.join('./data/', output_file)
# 如果路径不存在，则创建路径
if not os.path.exists(os.path.dirname(path)):
    os.makedirs(os.path.dirname(path))
with open(path, "w", encoding="utf-8") as f:
    f.write(json_str)